In [1]:
import os
import numpy as np
import pandas as pd
from PIL import Image
import cv2
import matplotlib.pyplot as plt
from sklearn.model_selection import StratifiedKFold
import wandb

import torch
from torch import nn
from torchvision import models
import torch.optim as optim

from dataset import get_dataloaders, get_datasets
from utils import seed_everything
from trainer import Trainer

# Params
Image.MAX_IMAGE_PIXELS = 1e11
CFG = {
    'seed': 42,
    'base_model': 'resnet34',   # resnet18/34/50, efficientnet_v2_s/m/l
    'img_size': 1024,
    'batch_size': 8,
    'freeze_epochs': 1,
    'epochs': 10,
    'base_lr': 1e-3,
    'affine_degrees': 10,
    'affine_translate': (0.1, 0.2),
    'affine_scale': (0.8, 1.2),
    'cv_fold': 5,
    'dataloader_num_workers': 36
}

# Wandb
wandb.login(key='1b0401db7513303bdea77fb070097f9d2850cf3b')
run = wandb.init(project='kaggle-ubc-ocean', config=CFG, tags=['torch', 'baseline'])

# Label encoder/decoder
encode = {'HGSC': 0, 'LGSC': 1, 'EC': 2, 'CC': 3, 'MC': 4}
decode = {v: k for k, v in encode.items()}

device = torch.device('cuda:0' if torch.cuda.is_available() else 'cpu')

# Paths
root = '/media/latlab/MR/projects/kaggle-ubc-ocean'
data_dir = os.path.join(root, 'data')
results_dir = os.path.join(root, 'results')
train_csv = 'train.csv'
train_image_dir = os.path.join(data_dir, 'train_images')
train_thumbnail_dir = os.path.join(data_dir, 'train_thumbnails')

# Seed
seed_everything(CFG['seed'])

# Load data
df = pd.read_csv(os.path.join(data_dir, train_csv))
df['label'] = df.loc[:,'label'].map(encode)

Failed to detect the name of this notebook, you can set it manually with the WANDB_NOTEBOOK_NAME environment variable to enable code saving.
wandb: Currently logged in as: naraiadam88. Use `wandb login --relogin` to force relogin
wandb: WARNING If you're specifying your api key in code, ensure this code is not shared publicly.
wandb: WARNING Consider setting the WANDB_API_KEY environment variable, or running `wandb login` from the command line.
wandb: Appending key for api.wandb.ai to your netrc file: /home/latlab/.netrc


In [2]:
def train_model(CFG, train_image_dir, train_thumbnail_dir, df_train, df_validation, encode, wandb_log=False):
    # Data loaders
    datasets = get_datasets(CFG, train_image_dir, train_thumbnail_dir, df_train, df_validation)
    dataloaders = get_dataloaders(CFG, datasets)

    # Model definition
    model = models.get_model(CFG['base_model'], weights='DEFAULT').to(device)
    for param in model.parameters():
        param.requires_grad = False
    model.fc = nn.Linear(model.fc.in_features, len(encode)).to(device)
    loss_fn = nn.CrossEntropyLoss()
    optimizer = optim.SGD(model.parameters(), lr=CFG['base_lr'], momentum=0.9)
    exp_lr_scheduler = optim.lr_scheduler.StepLR(optimizer, step_size=7, gamma=0.1)

    # Training
    trainer = Trainer(model, dataloaders, loss_fn, optimizer, exp_lr_scheduler, device, metric='balanced_accuracy', wandb_log=wandb_log)
    model, _ = trainer.train_epochs(num_epochs=CFG['freeze_epochs'])
    trainer.unfreeze()
    model, best_balanced_acc = trainer.train_epochs(num_epochs=CFG['epochs'])
    return model, best_balanced_acc

In [3]:
skf = StratifiedKFold(n_splits=CFG['cv_fold'], random_state=CFG['seed'], shuffle=True)
balanced_acc_list = []
lb = df.label
for cv, (train_index, valid_index) in enumerate(skf.split(np.zeros(len(lb)), lb)):
    print(f"Cross-validation fold {cv+1}/{CFG['cv_fold']}")
    df_train = df.iloc[train_index]
    df_validation = df.iloc[valid_index]
    run_name = f'{run.name}-cv{cv+1}'
    model, balanced_acc = train_model(CFG, train_image_dir, train_thumbnail_dir, df_train, df_validation, encode)
    balanced_acc_list.append(balanced_acc)
    torch.save(model.state_dict(), os.path.join(results_dir, 'models', f'ubc-ocean-{run_name}.pt'))
    wandb.log({f'balanced_acc_cv{cv+1}': balanced_acc})
wandb.log({f'mean_balanced_acc': np.mean(balanced_acc_list)})
wandb.finish()

Cross-validation fold 1/5
Epoch 1/1
----------


100%|██████████| 54/54 [00:08<00:00,  6.25it/s]


train loss: 1.5647, test loss: 1.4412, balanced_accuracy: 0.3670

Training complete in 0m 13s
Final balanced_accuracy: 0.366964

Epoch 1/10
----------


100%|██████████| 54/54 [00:13<00:00,  4.08it/s]


train loss: 1.3092, test loss: 1.2171, balanced_accuracy: 0.5760

Epoch 2/10
----------


...

train loss: 0.6206, test loss: 0.7520, balanced_accuracy: 0.7304

Epoch 9/10
----------


100%|██████████| 54/54 [00:12<00:00,  4.26it/s]


train loss: 0.6150, test loss: 0.7503, balanced_accuracy: 0.6935

Epoch 10/10
----------


100%|██████████| 54/54 [00:13<00:00,  4.04it/s]


train loss: 0.6042, test loss: 0.7275, balanced_accuracy: 0.7232

Training complete in 2m 54s
Final balanced_accuracy: 0.723214



In [4]:
# Final training on all data
model, _ = train_model(CFG, train_image_dir, train_thumbnail_dir, df, df, encode, wandb_log=False)
torch.save(model.state_dict(), os.path.join(results_dir, 'models', f'ubc-ocean-{run.name}.pt'))

Epoch 1/1
----------


100%|██████████| 68/68 [00:07<00:00,  8.60it/s]


train loss: 1.4656, test loss: 1.2588, balanced_accuracy: 0.3958

Training complete in 0m 17s
Final balanced_accuracy: 0.395752

Epoch 1/10
----------


100%|██████████| 68/68 [00:15<00:00,  4.42it/s]


train loss: 1.2973, test loss: 1.1971, balanced_accuracy: 0.4566

Epoch 2/10
----------


...

train loss: 0.5853, test loss: 0.4129, balanced_accuracy: 0.8468

Epoch 10/10
----------


100%|██████████| 68/68 [00:15<00:00,  4.28it/s]


train loss: 0.6010, test loss: 0.3729, balanced_accuracy: 0.8590

Training complete in 4m 13s
Final balanced_accuracy: 0.859007



wandb: WARNING Source type is set to 'repo' but some required information is missing from the environment. A job will not be created from this run. See https://docs.wandb.ai/guides/launch/create-job


balanced_accuracy,▁▂▄▅▆▆▇████
best_balanced_acc_cv1,▁
best_balanced_acc_cv2,▁
best_balanced_acc_cv3,▁
best_balanced_acc_cv4,▁
best_balanced_acc_cv5,▁
mean_best_balanced_acc,▁
train_loss,█▇▆▄▄▃▂▁▁▁▁
valid_loss,██▆▄▄▄▂▁▁▁▁
balanced_accuracy,0.85901
best_balanced_acc_cv1,0.66379
